<h1>EECS 549 Final Project</h1>

Index set of Reddit posts 

Get preliminary set of relevant documents from the BM25 baseline and a subset of random documents for manual annotation 

In [ ]:
import os 
from indexing import Indexer
from ranker import Ranker, BM25
from nltk import TweetTokenizer
import pickle
from tqdm import tqdm
import pandas as pd
import json
import numpy as np 

<h2>Index document set</h2>

In [ ]:
with open("../files/stopwords.txt") as f:
    words = f.read()
    stopwords = words.split()

In [ ]:
document_preprocessor = TweetTokenizer()

In [ ]:
index = Indexer.create_index('PositionalIndex', '../data/ask_reddit_posts_augmented.jsonl', document_preprocessor, stopwords, 10)

In [ ]:
with open("../files/index_pickle", "wb") as f: 
    pickle.dump(index, f)

<h2>Get BM25 Rankings</h2>

In [ ]:
df = pd.read_csv("../data/ask_reddit.csv")

In [ ]:
with open("../eval/simple_evaluation_queries.csv") as f:
    lines = f.readlines()
    queries = [line.strip().replace('"', '') for line in lines[1:]]

In [ ]:
bm25 = BM25(index)
ranker = Ranker(index, document_preprocessor, stopwords, bm25)

In [ ]:
docids = df['docid'].to_list()

In [ ]:
queries_to_rel = {}

for query in tqdm(queries):
    docs = ranker.query(query)
    ids = [doc[0] for doc in docs][:20]
    random_ids = list(np.random.choice(docids, 10))
    ids.extend(random_ids)
    queries_to_rel[query] = ids

<h2>Format data</h2>

In [ ]:
rel_df = pd.DataFrame.from_dict(queries_to_rel).T
rel_df.columns = [f"doc_{i}" for i in range(1, 31)]

In [ ]:
title_df = df[['title', 'id']]
for i in range(1, 31):
    target_col = rel_df[f'doc_{i}']

In [ ]:
rel_df.to_csv("../eval/bm25_relevance_baseline_queries.csv", header = None)

<h2>Look up docs</h2>

In [ ]:
docs = {}
with open("../data/ask_reddit_posts_augmented.jsonl", "r") as f:
    doc = f.readline()
    while doc:
        doc = json.loads(doc)
        id = doc['docid']
        content = doc['text']
        docs[id] = content
        doc = f.readline()

In [ ]:
np.random.choice(list(docs.keys()))


In [ ]:
docs[169243]